<font color = green >

## HOMEWORK - WORKING WITH REGEX

</font>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

<font color = green >

## Load TXT file 

</font>

In [4]:
import os
cwd= os.getcwd() # current working directory
# path = os.path.join(cwd,'data')
fn=  os.path.join('dates.txt')

with open(fn, 'r') as f:
    content = f.read()
print (content[:500])

df = pd.DataFrame(content.split('\n'), columns=['text'])
df['orig_index']=df.index
df['analyze1']=False
df.drop(500, axis='index', inplace=True)


03/25/93 Total time of visit (in minutes):
6/18/85 Primary Care Doctor:
sshe plans to move as of 7/8/71 In-Home Services: None
7 on 9/27/75 Audit C Score Current:
2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7
.Per 7/06/79 Movement D/O note:
4, 5/18/78 Patient's thoughts about current substance abuse:
10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview
3/7/86 SOS-10 Total Score:
(4/10/71)Score-1Audit C Score Current:
(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activ


<font color = green >

## Selecting text with dates

</font>

In [9]:
for cell_string in range(0,len(df)):
#    print('\n'+str(cell_string)+'\t'+df.iloc[cell_string, 0])
    t=re.findall(r'(\d{1,2}([\/ ]|[-])(\d{1,2}|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)([\/ ]|[-])\d{2,4})', df.iloc[cell_string, 0])
    if t!=[]:
        df.iloc[cell_string, 2]=t[0][0]
    else:
        t=re.findall(r'((January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[. ](\d{2}|[ ]\d{2})[, ](\d{2,4}|[ ]\d{2,4}))', df.iloc[cell_string, 0])
        if t!=[]: 
            df.iloc[cell_string, 2]=t[0][0]
        else:
            t=re.findall(r'((January|Janaury|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Decemeber|Dec)[, ](\d{2,4}|[ ]\d{2,4}))', df.iloc[cell_string, 0])
            if t!=[]: 
                df.iloc[cell_string, 2]=t[0][0]
            else:
                t=re.findall(r'\d{1,2}[\/]\d{4}|\d{4}', df.iloc[cell_string, 0])
                if t!=[]: 
                    df.iloc[cell_string, 2]=t[0]

        
print(500-df[df['analyze1']==False].count())
df.head()
# #--- df_view - Dataframe with records that need for datas recovering
# df_view=df[df['analyze1']==False]
# df_view.head()


text          500
orig_index    500
analyze1      500
dtype: int64


,text,orig_index,analyze1
0,03/25/93 Total time of visit (in minutes):,0,03/25/93
1,6/18/85 Primary Care Doctor:,1,6/18/85
2,sshe plans to move as of 7/8/71 In-Home Servic...,2,7/8/71
3,7 on 9/27/75 Audit C Score Current:,3,9/27/75
4,2/6/96 sleep studyPain Treatment Pain Level (N...,4,2/6/96


<font color = green >

## Converting recgnized data to required format

</font>

In [11]:
#----Отримання результату
import datetime as DT
from datetime import datetime

df['results']=False
for cell_string in range(0,len(df)):
    t=re.findall(r'\d{1,2}[\/-]\d{1,2}[\/-]\d{2}',df.loc[cell_string,'analyze1'])
    if t!=[]:
        t[0]=t[0].replace('-','/')
        df.loc[cell_string,'results']=datetime.strptime(t[0], '%m/%d/%y').date()
    else:
        t=re.findall('(\d{2}[ ](January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[ ]\d{4})', df.loc[cell_string, 'analyze1'])
        if t!=[]:
#            t[0]=t[0].replace('-','/')
            if len(t[0][0])==11: 
                df.loc[cell_string,'results']=datetime.strptime(t[0][0], '%d %b %Y').date()
            else:
                df.loc[cell_string,'results']=datetime.strptime(t[0][0], '%d %B %Y').date()
        else:
            t=re.findall('((January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)([.][ ]\d{2}|[ ]\d{2})([,][ ]\d{4}|[ ]\d{4}))', df.loc[cell_string, 'analyze1'])
            if t!=[]:
                temp=t[0][0].replace(',','')
                temp=temp.replace('.','')
                if len(temp)==11: 
                    df.loc[cell_string,'results']=datetime.strptime(temp, '%b %d %Y').date()
                else:
                    df.loc[cell_string,'results']=datetime.strptime(temp, '%B %d %Y').date()
            else:
                t=re.findall('((January|Janaury|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Decemeber|Dec)([,][ ]\d{4}|[ ]\d{4}))', df.loc[cell_string, 'analyze1'])
                if t!=[]:
                    temp=t[0][0].replace(',','')
                    temp=temp.replace('Janaury','January')
                    temp=temp.replace('Decemeber','December')
                    if len(temp)==8: 
                        df.loc[cell_string,'results']=datetime.strptime(temp, '%b %Y').date()
                    else:
                        df.loc[cell_string,'results']=datetime.strptime(temp, '%B %Y').date()
                else:
                    t=re.findall(r'\d{1,2}[\/]\d{4}|\d{4}', df.iloc[cell_string, 0])
                    if t!=[]:
                        if len(t[0])==4:
                            df.loc[cell_string,'results']=datetime.strptime(t[0], '%Y').date()
                        else:
                            df.loc[cell_string,'results']=datetime.strptime(t[0], '%m/%Y').date()

print(500-df[df['results']==False].count())
df.head()
#df_view=df[df['results']==False]
# df.head()

text          500
orig_index    500
analyze1      500
results       500
dtype: int64


,text,orig_index,analyze1,results
0,03/25/93 Total time of visit (in minutes):,0,03/25/93,1993-03-25
1,6/18/85 Primary Care Doctor:,1,6/18/85,1985-06-18
2,sshe plans to move as of 7/8/71 In-Home Servic...,2,7/8/71,1971-07-08
3,7 on 9/27/75 Audit C Score Current:,3,9/27/75,1975-09-27
4,2/6/96 sleep studyPain Treatment Pain Level (N...,4,2/6/96,1996-02-06


<font color = green >

## Sorting and preparing final result

</font>

In [13]:
#------Sorting---
df.sort_values('results', inplace=True)
df=df[['results','orig_index','text']]
df=df.reset_index()
df.drop(['index'], axis=1, inplace=True)
print(df[['results','orig_index']])

        results  orig_index
0    1971-04-10           9
1    1971-05-18          84
2    1971-07-08           2
3    1971-07-11          53
4    1971-09-12          28
..          ...         ...
495  2020-05-12          34
496  2020-07-18         102
497  2020-07-20         109
498  2020-09-14          83
499  2020-12-05          58

[500 rows x 2 columns]
